In [1]:
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [2]:
airports = [
    "KATL",
    "KCLT",
    "KDEN",
    "KDFW",
    "KJFK",
    "KMEM",
    "KMIA",
    "KORD",
    "KPHX",
    "KSEA",
]

In [82]:
airport = airports[9]
airport

'KSEA'

In [83]:
!pwd

/home/charmichokshi4444/Data


In [84]:
# !tar -xvf KSEA.tar

In [85]:
DATA_DIRECTORY = Path("./")


In [86]:
DATA_DIRECTORY

PosixPath('.')

In [87]:
# airport = "KSEA"
pushback = pd.read_csv(DATA_DIRECTORY / f"train_labels_{airport}.csv.bz2")
# pushback

In [88]:
etd = pd.read_csv(
    DATA_DIRECTORY / airport / f"{airport}_etd.csv.bz2",
    parse_dates=["departure_runway_estimated_time", "timestamp"],
).sort_values("timestamp")
# etd

# get the etd for each pushback line

In [89]:
pushback.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647433 entries, 0 to 1647432
Data columns (total 4 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   gufi                    1647433 non-null  object
 1   timestamp               1647433 non-null  object
 2   airport                 1647433 non-null  object
 3   minutes_until_pushback  1647433 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 50.3+ MB


In [90]:
cp = pushback
# cpp = cp.iloc[0:15]
cp['timestamp'] = pd.to_datetime(cp['timestamp'])
# # cpp.info()

In [91]:
cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647433 entries, 0 to 1647432
Data columns (total 4 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   gufi                    1647433 non-null  object        
 1   timestamp               1647433 non-null  datetime64[ns]
 2   airport                 1647433 non-null  object        
 3   minutes_until_pushback  1647433 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 50.3+ MB


In [92]:
def estimate_pushback(now: pd.Timestamp) -> pd.Series:

    # subset submission format to the current prediction time
    now_submission_format = cp.loc[
        cp.timestamp == now
    ].reset_index(drop=True)

    # filter features to 30 hours before prediction time to prediction time
    now_etd = etd.loc[(etd.timestamp > now - timedelta(hours=30)) & (etd.timestamp <= now)]

    # get the latest ETD for each flight
    latest_now_etd = now_etd.groupby("gufi").last().departure_runway_estimated_time

    # merge the latest ETD with the flights we are predicting
    departure_runway_estimated_time = now_submission_format.merge(
        latest_now_etd, how="left", on="gufi"
    ).departure_runway_estimated_time

    now_prediction = now_submission_format.copy()

    now_prediction["departure_runway_estimated_time"] = departure_runway_estimated_time

    return now_prediction

predictions = process_map(
    estimate_pushback,
    pd.to_datetime(cp.timestamp.unique()),
    chunksize=20,
)

predictions = pd.concat(predictions, ignore_index=True)
predictions["departure_runway_estimated_time"] = predictions.departure_runway_estimated_time

predictions.info()

  0%|          | 0/50438 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1647433 entries, 0 to 1647432
Data columns (total 5 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   gufi                             1647433 non-null  object        
 1   timestamp                        1647433 non-null  datetime64[ns]
 2   airport                          1647433 non-null  object        
 3   minutes_until_pushback           1647433 non-null  int64         
 4   departure_runway_estimated_time  1647433 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 62.8+ MB


In [93]:
predictions.to_csv('{}_etd.csv'.format(airport), index=False)